In [4]:
import numpy as np
import awkward as ak
from coffea import processor
import json
import hist
from coffea.nanoevents import NanoEventsFactory, BaseSchema, PFNanoAODSchema
import coffea.nanoevents.methods.vector as vector
import warnings
import matplotlib.pyplot as plt
from lpcjobqueue import LPCCondorCluster
from distributed import Client
import fastjet

In [2]:
warnings.filterwarnings("ignore", "Found duplicate branch")
warnings.filterwarnings("ignore", "Missing cross-reference index for")
warnings.filterwarnings("ignore", "dcut")

In [ ]:
cluster = LPCCondorCluster(ship_env=True)
cluster.adapt(minimum=0, maximum=75)
client = Client(cluster)

In [3]:
#with open("jsons/qcd_and_more_hj_files.json") as fin:
with open("jsons/600-800.json") as fin:
    filesets = json.load(fin)

In [ ]:
class MyProcessor(processor.ProcessorABC):
    
    def __init__(self):
        pass
    
    def process(self, events):
        dataset = events.metadata['dataset']
        
        fatjet = events.FatJet
        pt_cut = (fatjet.pt > 300)
        boosted_fatjet = fatjet[pt_cut]
        
        def color_ring(fatjet):
            jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 1.0)
            pf = ak.flatten(fatjet.constituents.pf, axis=1)
            cluster = fastjet.ClusterSequence(pf, jetdef)
            subjets = cluster.exclusive_subjets_up_to(data=cluster.exclusive_jets(n_jets=1), nsub=3)
            vec = ak.zip({
                "x": subjets.px,
                "y": subjets.py,
                "z": subjets.pz,
                "t": subjets.E,
                },
                with_name = "LorentzVector",
                behavior=vector.behavior,
                )
            vec = ak.pad_none(vec, 3)
            vec["norm3"] = np.sqrt(vec.dot(vec))
            i, j = ak.unzip(ak.combinations(vec, 2))
            best = ak.argmax((i + j).mass, axis=1, keepdims=True)
            leg1, leg2 = i[best], j[best]
            #assert ak.all((leg1 + leg2).mass == ak.max((i + j).mass, axis=1))
            leg3 = vec[(best == 0)*2 + (best == 1)*1 + (best == 2)*0]
            #assert ak.all(leg3.x != leg1.x)
            #assert ak.all(leg3.x != leg2.x)
            a12 = np.arccos(leg1.dot(leg2) / (leg1.norm3 * leg2.norm3))
            a13 = np.arccos(leg1.dot(leg3) / (leg1.norm3 * leg3.norm3))
            a23 = np.arccos(leg2.dot(leg3) / (leg2.norm3 * leg3.norm3))
            color_ring = ((a13**2 + a23**2)/(a12**2))
            return color_ring
        uf_cr = ak.unflatten(ak.flatten(color_ring(boosted_fatjet)), counts=ak.num(boosted_fatjet))
        boosted_fatjet['color_ring'] = uf_cr
        
        hcr = (
            hist.Hist.new
            .Reg(40, 0, 10, name='color_ring', label='Color_Ring')
            .Double()
        )
        
        #cut = (fatjet.pt > 450) & (fatjet.btagDeepB < 0.4941) & (fatjet.color_ring <= 1)
        #subset = fatjet[cut]
        fill_cr = ak.fill_none(ak.flatten(boosted_fatjet.color_ring), 99999)
        hcr.fill(color_ring=fill_cr)
        
        return {
            dataset: {
                "entries": len(events),
                "Color_Ring": hcr,
            }
        }
    
    def postprocess(self, accumulator):
        pass

In [ ]:
processor_instance=MyProcessor()
futures_run = processor.Runner(
    #executor = processor.FuturesExecutor(compression=None, workers=8),
    executor = processor.DaskExecutor(client=client),
    schema=PFNanoAODSchema,
    #skipbadfiles=True,
    #maxchunks=10,
    #chunksize=1000,
)

out = futures_run(
    filesets,
    "Events",
    processor_instance=MyProcessor()
)
out

In [ ]:
fig, ax = plt.subplots()
out['Hbb']['Color_Ring'].plot1d(ax=ax)
plt.xlim(0,1)

In [ ]:
filesets["QCD_Pt_470to600_TuneCP5_13TeV_pythia8"][-1]

In [ ]:
events = NanoEventsFactory.from_root(
    filesets["QCD_Pt_470to600_TuneCP5_13TeV_pythia8"][-1],
    schemaclass=PFNanoAODSchema,
).events()

In [ ]:
fatjet = events.FatJet
pt_cut = (fatjet.pt > 300)
boosted_fatjet = fatjet[pt_cut]

In [5]:
def color_ring(fatjet):
    jetdef = fastjet.JetDefinition(fastjet.antikt_algorithm, 1.0)
    pf = ak.flatten(fatjet.constituents.pf, axis=1)
    cluster = fastjet.ClusterSequence(pf, jetdef)
    subjets = cluster.exclusive_subjets_up_to(data=cluster.exclusive_jets(n_jets=1), nsub=3)
    vec = ak.zip({
        "x": subjets.px,
        "y": subjets.py,
        "z": subjets.pz,
        "t": subjets.E,
        },
        with_name = "LorentzVector",
        behavior=vector.behavior,
        )
    vec = ak.pad_none(vec, 3)
    vec["norm3"] = np.sqrt(vec.dot(vec))
    i, j = ak.unzip(ak.combinations(vec, 2))
    best = ak.argmax((i + j).mass, axis=1, keepdims=True)
    leg1, leg2 = i[best], j[best]
    #assert ak.all((leg1 + leg2).mass == ak.max((i + j).mass, axis=1))
    leg3 = vec[(best == 0)*2 + (best == 1)*1 + (best == 2)*0]
    #assert ak.all(leg3.x != leg1.x)
    #assert ak.all(leg3.x != leg2.x)
    a12 = np.arccos(leg1.dot(leg2) / (leg1.norm3 * leg2.norm3))
    a13 = np.arccos(leg1.dot(leg3) / (leg1.norm3 * leg3.norm3))
    a23 = np.arccos(leg2.dot(leg3) / (leg2.norm3 * leg3.norm3))
    color_ring = ((a13**2 + a23**2)/(a12**2))
    return color_ring

In [ ]:
uf_cr = ak.unflatten(ak.flatten(color_ring(boosted_fatjet)), counts=ak.num(boosted_fatjet))
boosted_fatjet['color_ring'] = uf_cr

In [ ]:
hcr = (
            hist.Hist.new
            .Reg(20, 0, 50, name='color_ring', label='Color_Ring')
            .Double()
        )

In [ ]:
fill_cr = ak.fill_none(ak.flatten(boosted_fatjet.color_ring), 0)

In [ ]:
hcr.fill(color_ring=fill_cr)

In [ ]:
import warnings
#warnings.filterwarnings("error", "invalid value encountered in sqrt")
bad = []
for i in range(len(filesets["QCD_Pt_600to800_TuneCP5_13TeV_pythia8"])):
#for i in range(10,15):
    try:
        events = NanoEventsFactory.from_root(
            filesets["QCD_Pt_600to800_TuneCP5_13TeV_pythia8"][i],
            schemaclass=PFNanoAODSchema,
        ).events()
        fatjet = events.FatJet
        pt_cut = (fatjet.pt > 300)
        boosted_fatjet = fatjet[pt_cut]
        uf_cr = ak.unflatten(ak.flatten(color_ring(boosted_fatjet)), counts=ak.num(boosted_fatjet))
        boosted_fatjet['color_ring'] = uf_cr
        hcr = (
                hist.Hist.new
                .Reg(20, 0, 50, name='color_ring', label='Color_Ring')
                .Double()
            )
        fill_cr = ak.fill_none(ak.flatten(boosted_fatjet.color_ring), 99999)
        hcr.fill(color_ring=fill_cr)
    except:
        bad.append(filesets["QCD_Pt_600to800_TuneCP5_13TeV_pythia8"][i])
    if i % 20 == 0:
        print(i)

#--------------------------------------------------------------------------
#                         FastJet release 3.4.0
#                 M. Cacciari, G.P. Salam and G. Soyez                  
#     A software package for jet finding and analysis at colliders      
#                           http://fastjet.fr                           
#	                                                                      
# Please cite EPJC72(2012)1896 [arXiv:1111.6097] if you use this package
# for scientific work and optionally PLB641(2006)57 [hep-ph/0512210].   
#                                                                       
# FastJet is provided without warranty under the GNU GPL v2 or higher.  
# It uses T. Chan's closest pair algorithm, S. Fortune's Voronoi code,
# CGAL and 3rd party plugin jet algorithms. See COPYING file for details.
#--------------------------------------------------------------------------
0
20
40


60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420


In [ ]:
bad

In [ ]:
with open('600-800_bad.txt','w') as f:
    for i in bad:
        f.write(i+'\n')

In [ ]:
events.FatJet

In [ ]:
color_ring(boosted_fatjet)

In [ ]:
ak.flatten(boosted_fatjet)